In [1]:
pip install --upgrade langchain-core

    PyYAML (>=5.1.*)
            ~~~~~~^
Note: you may need to restart the kernel to use updated packages.


In [5]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

KeyboardInterrupt: Interrupted by user

In [6]:
from typing import Optional

from pydantic import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    # ^ Doc-string for the entity Person.
    # This doc-string is sent to the LLM as the description of the schema Person,
    # and it can help to improve extraction results.

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    name: Optional[str] = Field(default=None, description="The name of the person")
    hair_color: Optional[str] = Field(
        default=None, description="The color of the person's hair if known"
    )
    height_in_meters: Optional[str] = Field(
        default=None, description="Height measured in meters"
    )

In [7]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Define a custom prompt to provide instructions and any additional context.
# 1) You can add examples into the prompt template to improve extraction quality
# 2) Introduce additional parameters to take context into account (e.g., include metadata
#    about the document from which the text was extracted.)
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",
        ),
        # Please see the how-to about improving performance with
        # reference examples.
        # MessagesPlaceholder('examples'),
        ("human", "{text}"),
    ]
)

In [2]:
pip install -qU "langchain[openai]"

    PyYAML (>=5.1.*)
            ~~~~~~^
Note: you may need to restart the kernel to use updated packages.


In [8]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [9]:
structured_llm = llm.with_structured_output(schema=Person)

In [10]:
text = "Alan Smith is 6 feet tall and has blond hair."
prompt = prompt_template.invoke({"text": text})
structured_llm.invoke(prompt)

/Users/irisliu/anaconda3/lib/python3.10/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=9f29a65a-73ad-447a-9540-c0709e1c245c,id=9f29a65a-73ad-447a-9540-c0709e1c245c
Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=9f29a65a-73ad-447a-9540-c0709e1c245c,id=9f29a65a-73ad-447a-9540-c0709e1c245c; trace=4efd5956-87ee-46b2-abfc-0e19b0708380,id=4efd5956-87ee-46b2-abfc-0e19b0708380; trace=4efd

Person(name='Alan Smith', hair_color='blond', height_in_meters='1.83')

Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=4efd5956-87ee-46b2-abfc-0e19b0708380,id=cc86d0af-8bb1-4482-a473-ad79f0687997; trace=4efd5956-87ee-46b2-abfc-0e19b0708380,id=4761a307-4206-4393-8669-7cf622d7f3dd; trace=4efd5956-87ee-46b2-abfc-0e19b0708380,id=4761a307-4206-4393-8669-7cf622d7f3dd; trace=4efd5956-87ee-46b2-abfc-0e19b0708380,id=4efd5956-87ee-46b2-abfc-0e19b0708380


In [11]:
from typing import List, Optional

from pydantic import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    # ^ Doc-string for the entity Person.
    # This doc-string is sent to the LLM as the description of the schema Person,
    # and it can help to improve extraction results.

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    name: Optional[str] = Field(default=None, description="The name of the person")
    hair_color: Optional[str] = Field(
        default=None, description="The color of the person's hair if known"
    )
    height_in_meters: Optional[str] = Field(
        default=None, description="Height measured in meters"
    )


class Data(BaseModel):
    """Extracted data about people."""

    # Creates a model so that we can extract multiple entities.
    people: List[Person]

In [12]:
structured_llm = llm.with_structured_output(schema=Data)
text = "My name is Jeff, my hair is black and i am 6 feet tall. Anna has the same color hair as me."
prompt = prompt_template.invoke({"text": text})
structured_llm.invoke(prompt)

Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=e341f64f-b516-492a-ba74-f238828f6856,id=e341f64f-b516-492a-ba74-f238828f6856; trace=e341f64f-b516-492a-ba74-f238828f6856,id=e341f64f-b516-492a-ba74-f238828f6856; trace=97aa2e5a-25d4-4992-993b-194df6fa151c,id=97aa2e5a-25d4-4992-993b-194df6fa151c; trace=97aa2e5a-25d4-4992-993b-194df6fa151c,id=38d6853b-9e32-4300-a6bb-ecf4daa6f4a0


Data(people=[Person(name='Jeff', hair_color='black', height_in_meters='1.83'), Person(name='Anna', hair_color='black', height_in_meters=None)])

Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=97aa2e5a-25d4-4992-993b-194df6fa151c,id=38d6853b-9e32-4300-a6bb-ecf4daa6f4a0; trace=97aa2e5a-25d4-4992-993b-194df6fa151c,id=7cfcf2f8-85f7-4ba5-bcbd-76fa4c8cd069; trace=97aa2e5a-25d4-4992-993b-194df6fa151c,id=7cfcf2f8-85f7-4ba5-bcbd-76fa4c8cd069; trace=97aa2e5a-25d4-4992-993b-194df6fa151c,id=97aa2e5a-25d4-4992-993b-194df6fa151c
